In [1]:
import pandas as pd
import re
import operator
import calendar
import urllib.request, urllib.error, urllib.parse
from datetime import datetime
from bs4 import BeautifulSoup
# Get Tourneys
fileName=('BUWomensRecordBook.txt')
tourneys=[]
teamSet=set()
with open(fileName, 'r', encoding='utf-8') as f:
    read_data = f.read()
    rows=read_data.split('\n')
    conf='Independent'
    tourneyDict={}
    for i in rows:
        row=i.split('  ')
        if(len(row)==2):
            tourneyDict[row[0]]=row[1]
            #print(row)
            

In [2]:
# Get Games
fileName=('BUWomensRecordBook.txt')
gameList=[]
with open(fileName, 'r', encoding='utf-8') as f:
    read_data = f.read()
    rows=read_data.split('\n')
    conf='Independent'
    for i in rows:      
        if(len(i)==7):
            season=i 
        coachSearch=re.search("COACH: (.*)",i)
        if coachSearch != None:
            coach=coachSearch.group(1)
        captSearch=re.search("CAPTAIN.?: (.*)",i)
        if captSearch != None:
            capt=captSearch.group(1) 
            #print(capt)
        confSearch=re.search("(NEIHL|ECAC|HOCKEY EAST):",i)
        if confSearch != None:
            conf=confSearch.group(1)
            if(conf=='HOCKEY EAST'):
                conf='Hockey East'
        if(re.search('\*',i)!=None):
            gameType='Conference'
        else:
            gameType='Non-Conference'
        i=i.replace("* ",'')
        note=''
        if('†' in i):
            note='Loss by forfeit (ineligible player)'
            i=i.replace('†','')
        if('+' in i):
            note='Win by forfeit (opponent left ice – score was 5-1, BU)'
            i=i.replace('+','')
        if('‡' in i):
            note='Win by forfeit (ineligible player)'
            i=i.replace('‡','')
        game=re.search(r"(\d*\/\d*) (\w) (?:\((.?ot)\))? ?(.*) (\w) (#\d*)? ?(\S*|\S* \S*|\S* \S* \S*) ?(\(.*\))? (\d*-\d*)",i)
        if game==None:
            continue
        #print(game.groups())
        gameDict={'date':game.group(1),
                 'result':game.group(2),
                 'ot':game.group(3),
                 'arena':game.group(4),
                 'opponent':game.group(7),
                 'gameType':gameType,
                 'tourney': game.group(8),
                 'scoreline':game.group(9),
                 'location':game.group(5),
                 'rank':game.group(6),
                 'coach':coach,
                 'captain':capt,
                 'conference':conf,
                 'season':season,
                 'note':note}
        if(gameDict['gameType']==None):
            gameDict['gameType']='Non-Conference'
        if(gameDict['tourney']=='(ex)' or gameDict['result']=='E'):
            gameDict['gameType']='Exhibition'
            gameDict['result'] = 'E'
            gameDict['tourney'] = None

        if(gameDict['tourney']!=None):
            #print(gameDict)
            gameDict['tourney']=tourneyDict[gameDict['tourney'].replace('(','').replace(')','')]
        if((gameDict['tourney'] == gameDict['conference'] +" "+ 'Tournament') or (gameDict['tourney'] == 'NCAA Tournament')):
            gameDict['seasonType'] = 'Playoffs'
        else:
            gameDict['seasonType'] = 'Regular Season'
        if(gameDict['location']=='H'):
            gameDict['location']='Home'
        elif(gameDict['location']=='A'):
            gameDict['location']='Away'
        elif(gameDict['location']=='N'):
            gameDict['location']='Neutral'
        gameDict['month'],gameDict['day']=gameDict['date'].split('/')
        gameDict['month']=int(gameDict['month'])
        gameDict['day']=int(gameDict['day'])
        if(gameDict['month'] >=9):
            gameDict['date']+="/" + gameDict['season'][:4]
            gameDict['year']=int(gameDict['season'][:4])
        elif(gameDict['month'] <= 4):
            gameDict['date']+= "/" + str(int(gameDict['season'][:4])+1)
            gameDict['year']=int(gameDict['season'][:4])+1
        gameDict['BUScore'],gameDict['OppoScore']=int(gameDict['scoreline'].split('-')[0]),int(gameDict['scoreline'].split('-')[1])
        gameDict['GD']=abs(gameDict['BUScore']-gameDict['OppoScore'])
        gameDict['date']=pd.Timestamp(gameDict['date'])
        gameDict['dow']=gameDict['date'].weekday()
        gameList.append(gameDict)
f.close()
dfWomensGames=pd.DataFrame(gameList)

In [3]:
import urllib.request, urllib.error, urllib.parse
from bs4 import BeautifulSoup
import re

skaterSeasonList=[]
goalieSeasonList=[]
#seasons=["2002-03","2003-04","2004-05","2005-06","2006-07","2007-08","2008-09","2009-10","2010-11","2011-12","2012-13","2013-14","2014-15","2015-16","2016-17","2017-18","2018-19","2019-20","2020-21"]
seasons=["2014-15","2015-16","2016-17","2017-18","2018-19","2019-20","2020-21"]

for season in seasons:
    sSeason=season[2:4]+season[-2:]
    url = "http://www.collegehockeystats.net/{}/teamstats/bu_w".format(sSeason)
    #print(url)
    f=urllib.request.urlopen(url)
    html = f.read()
    f.close()
    soup = BeautifulSoup(html, 'html.parser')
    tab = soup.find_all("table")
    skatersTab=tab[2]
    tabRows=skatersTab.find_all('tr')
    statsDict={"number" : 0,
    "name" : 1,
    "pos" : 2,
    "yr" : 3,
    "overall_gp" : 4,
    "overall_goals" : 5,
    "overall_assists" : 6,
    "overall_pts" : 7,
    "overall_pen" : 8,
    "overall_ppg" : 9,
    "overall_shg" : 10,
    "overall_gwg" : 11,
    "overall_plusminus" : 12,
    "overall_sog" : 13,
    "conf_gp" : 14,
    "conf_goals" : 15,
    "conf_assists" : 16,
    "conf_pts" : 17,
    "conf_pen" : 18,
    "conf_ppg" : 19,
    "conf_shg" : 20,
    "conf_gwg" : 21,
    "conf_plusminus" : 22,
    "conf_sog" : 23,
    "career_gp" : 24,
    "career_goals" : 25,
    "career_assists" : 26,
    "career_pts" : 27}

    gStatsDict={"number" : 0,
    "name" : 1,
    "yr" : 2,
    "gp" : 3,
    "mins" : 4,
    "ga" : 5,
    "saves" : 6,
    "shots" : 7,
    "saveperc" : 8,
    "gaa" : 9,
    "record" : 10,
    "winpec" : 11,
    "gs" : 12,
    "so" : 13}

    skateRows=skatersTab.find_all('tr', class_=lambda x: x != 'chssmallbold')
    for i in range(1,len(skateRows)-1):
        playerDict={}
        for d in statsDict.keys():
                playerDict[d] = skateRows[i].find_all('td')[statsDict[d]].get_text().lstrip(' ').rstrip(' ').replace('\xa0','N/A')
        playerDict['name']=re.sub(' \(.*','',playerDict['name'])
        playerDict['season']=season

        skaterSeasonList.append(playerDict)


    goalieTab=tab[3]
    goalieRows=goalieTab.find_all('tr', class_=lambda x: x != 'chssmallbold')

    for i in range(1,len(goalieRows)):
        playerDict={}
        if("Open Net" in goalieRows[i].get_text() or "##" in goalieRows[i].get_text()):
            break
        for d in gStatsDict.keys():
            playerDict[d] = goalieRows[i].find_all('td')[gStatsDict[d]].get_text().lstrip(' ').rstrip(' ').replace('\xa0','N/A')
        playerDict['name']=re.sub(' \(.*','',playerDict['name'])
        playerDict['season']=season
        goalieSeasonList.append(playerDict)



In [4]:
dfSkateNew=pd.DataFrame(skaterSeasonList)

In [5]:
dfGoalNew=pd.DataFrame(goalieSeasonList)

In [6]:
seasons=["2005-06","2006-07","2007-08","2008-09","2009-10","2010-11","2011-12","2012-13","2013-14"]
#seasons=["2014-15","2015-16","2016-17","2017-18","2018-19","2019-20","2020-21"]
playerList=[]
goalieList=[]
for season in seasons:
    sSeason=season[2:4]+season[-2:]
    url = "http://www.collegehockeystats.net/{}/teamstats/bu_w".format(sSeason)
    #print(url)
    f=urllib.request.urlopen(url)
    html = f.read()
    f.close()
    soup = BeautifulSoup(html, 'html.parser')
    table= soup.find("pre")
    rows=table.get_text().split('\n')
    goalies=False
    for i in rows:
        if('Goaltending (Overall)' in i):
            goalies=True
        if('Goaltending (Conference Only)' in i):
            break
        if('--' not in i and 'Boston University' not in i and 'Opponents' not in i and '##' not in i and 'Bench' not in i and not goalies):
            name,overall,conf,career=i.split('|')
            name=re.sub('\(.*\)','',name)
            name=name.split(' ')
            overall=overall.split(' ')
            name=list(filter(None,name))
            overall=list(filter(None,overall))
            if(overall[4]=='0'):
                overall[4]+='/'
            playDict={'number':name[0],
                      'name':name[1]+ " " +name[2],
                      'pos':name[3],
                      'yr':name[4],
                      'gp':overall[0],
                      'goals':overall[1],
                      'assists':overall[2],
                      'pts':overall[3],
                      'pen':overall[4]+overall[5],
                      'season':season}
            playerList.append(playDict)
        if('----' not in i and 'Boston University' not in i and 'Opponents' not in i and '##' not in i and 'Open' not in i and goalies):
            name,overall,none=i.split('|')
            name=re.sub('\(.*\)','',name)
            name=name.split(' ')
            overall=overall.split(' ')
            name=list(filter(None,name))
            overall=list(filter(None,overall))
            if(len(overall)==13):
                goalieDict={'number':name[0],
                          'name':name[1]+ " " +name[2],
                          'yr':name[3],
                          'gp':overall[0],
                          'mins':overall[1],
                          'ga':overall[2],
                          'saves':overall[3],
                          'sv%':overall[5],
                          'gaa':overall[6],
                          'record':overall[7]+overall[8],
                          'SO':overall[11],
                          'season':season}
            else:
                goalieDict={'number':name[0],
                      'name':name[1]+ " " +name[2],
                      'yr':name[3],
                      'gp':overall[0],
                      'mins':overall[1],
                      'ga':overall[2],
                      'saves':overall[3],
                      'sv%':overall[5],
                      'gaa':overall[6],
                      'record':overall[7]+overall[8]+overall[9],
                      'SO':overall[11],
                      'season':season}
            goalieList.append(goalieDict)
dfPlay=pd.DataFrame(playerList)
dfGoal=pd.DataFrame(goalieList)

KeyboardInterrupt: 

In [ ]:
for i in dfPlay.to_string(index=False).split('\n'):
    row=list(filter(None,i.split('  ')))
    print("\t".join(row))

In [ ]:
dfSkate=dfSkateNew[['number','name','pos','yr','overall_gp','overall_goals','overall_assists','overall_pts','overall_pen','season']]

In [29]:
for i in dfGoal.to_string(index=False).split('\n'):
    row=list(filter(None,i.split('  ')))
    print("\t".join(row))

number	 name	yr	gp	 mins	ga saves	sv%	 gaa	 record	SO	 season
39	Allyse Wilcox	FR	28	1597:19	70	 736	 .913	2.63	 9-14-4	 1	2005-06
23	Karilyn Pilch	FR	 8	 395:52	24	 184	 .885	3.64	3-3-0	 6	2005-06
 1	 Eden Spencer	FR	 1	 3:03	 0	 2	1.000	0.00	0-0-0	 0	2005-06
39	Allyse Wilcox	SO	20	1168:48	48	 520	 .915	2.46	 10-9-1	20	2006-07
33	Melissa Haber	FR	15	 879:25	31	 312	 .910	2.12	9-3-2	14	2006-07
39	Allyse Wilcox	JR	26	1534:16	50	 613	 .925	1.96	12-12-2	 3	2007-08
33	Melissa Haber	SO	10	 551:21	36	 271	 .883	3.92	3-5-1	 9	2007-08
 1	Ashley Leichliter	SO	 2	31:29	 0	 5	1.000	0.00	0-0-0	 0	2007-08
39	Allyse Wilcox	SR	24	1436:15	54	 549	 .910	2.26	 12-8-4	24	2008-09
33	Melissa Haber	JR	12	 732:02	20	 258	 .928	1.64	6-3-3	12	2008-09
 1	Ashley Leichliter	JR	 2	15:09	 2	 4	 .667	7.92	0-0-0	 0	2008-09
33	Melissa Haber	SR	32	1906:23	65	 773	 .922	2.05	 15-8-8	32	2009-10
30	 Alissa Fromkin	FR	 8	 439:17	19	 143	 .883	2.60	2-1-4	 6	2009-10
 1	Kerrin Sperry	FR	28	1689:52	46	 601	 .929	1.63	 21-4-3	2

In [36]:
dfGN=dfGoalNew[['number','name','yr','gp','mins','ga','saves','saveperc','gaa','record','so','season']]

In [37]:
for i in dfGN.to_string(index=False).split('\n'):
    row=list(filter(None,i.split('  ')))
    print("\t".join(row))

number	 name	yr	gp	 mins	ga saves saveperc	 gaa	record so	 season
33	Victoria Hanson	SO	23	1332:17	50	 506	 .910	2.25	16-5-2	0	2014-15
31	Erin O'Neil	FR	16	 870:00	39	 362	 .903	2.69	 9-4-1	0	2014-15
30	 Mia Becker	SO	 2	23:04	 2	12	 .857	5.20	 0-0-0	0	2014-15
31	Erin O'Neil	SO	30	1671:13	81	 809	 .909	2.91	17-8-2	2	2015-16
33	Victoria Hanson	JR	12	 655:53	35	 278	 .888	3.20	 6-6-0	0	2015-16
 1	Ashlyn Aiello	SO	 2	11:36	 0	 6	1.000	0.00	 0-0-0	0	2015-16
33	Victoria Hanson	SR	28	1693:17	67	 845	 .927	2.37	15-9-4	2	2016-17
31	Erin O'Neil	JR	 9	 547:12	34	 275	 .890	3.73	 4-3-2	0	2016-17
30	Corinne Schroeder	FR	25	1436:17	60	 633	 .913	2.51	12-8-4	0	2017-18
31	Erin O'Neil	SR	14	 805:50	35	 412	 .922	2.61	 2-9-2	1	2017-18
 1	Ashlyn Aiello	SR	 1	 6:57	 0	 5	1.000	0.00	 0-0-0	0	2017-18
30	Corinne Schroeder	SO	32	1930:17	61	 846	 .933	1.90	18-7-7	2	2018-19
 1	Kate Stuart	FR	 5	 303:38	 9	 149	 .943	1.78	 3-1-1	0	2018-19
31	Nicole McGuigan	FR	 2	29:11	 0	 3	1.000	0.00	 0-0-0	0	2018-19
30	Corin

In [31]:
url = "https://goterriers.com/sports/womens-ice-hockey/stats/?path=whockey"
#print(url)
f=urllib.request.urlopen(url)
html = f.read()
f.close()
soup = BeautifulSoup(html, 'html.parser')
curSkate=soup.find('section',{'id':'individual-overall-skaters'})
rows=curSkate.find_all('tr')
currSkaters=[]
for i in rows:
    col=i.find_all('td')
    name=i.find('span')
    if(name!=None):
        counter=0
        if('Team' in name):
            break
        lastName,firstName=name.get_text().split(', ')
        skateDict={'number':int(col[0].get_text()),
                   'last':lastName,
                   'first':firstName,
                   'name':firstName+' '+lastName,
                    'gp':int(col[1].get_text()),
                    'goals':int(col[2].get_text()),
                    'assists':int(col[3].get_text()),
                    'pts':int(col[4].get_text()),
                    'pen':col[16].get_text().replace('-','/')}
        currSkaters.append(skateDict)
dfCurr=pd.DataFrame(currSkaters)


In [32]:
dfC= dfCurr[['number','name','gp','goals','assists','pts','pen']]
for i in dfC.to_string(index=False).split('\n'):
    row=list(filter(None,i.split('  ')))
    print("\t".join(row))

 number	name	gp	goals	assists	pts	pen
5	Courtney Correia	29	 13	9	 22	4/8
 16	Julia Nearis	28	4	 16	 20	 7/14
 11	 Mackenna Parker	30	9	7	 16	10/20
 17	Lacey Martin	28	5	7	 12	3/6
 28	Catherine Foulem	31	4	8	 12	4/8
6	 Nadia Mattivi	27	3	9	 12	14/28
 20	 Julia Shaunessy	26	4	6	 10	16/32
 19	 Emma Wuthrich	31	5	4	9	11/22
 18	Haylee Blinkhorn	30	3	5	8	1/2
 22	Christina Vote	31	5	2	7	 7/14
2	Alex Allan	31	3	4	7	 6/12
 15	Ellie Larson	24	2	5	7	0/0
7	 Kylie Roberts	14	4	2	6	1/2
 13	Andi Calderone	31	0	6	6	4/8
9	 Clare O'Leary	24	1	4	5	1/2
 10	Jesse DeVito	27	0	5	5	 8/16
 23	Tamara Giaquinto	25	0	5	5	1/2
 12	Kaleigh Donnelly	26	1	3	4	1/2
5	Courtney Correia	 2	1	2	3	0/0
 21	 Madison Cardaci	17	0	3	3	4/8
 11	 Mackenna Parker	 2	1	1	2	1/2
 26	Brooke Ersoy	17	0	2	2	3/6
 20	 Julia Shaunessy	 2	0	2	2	1/2
 16	Julia Nearis	 2	1	0	1	0/0
 19	 Emma Wuthrich	 2	1	0	1	2/4
 21	 Madison Cardaci	 2	0	1	1	0/0
 23	Tamara Giaquinto	 1	0	1	1	0/0
2	Alex Allan	 2	0	0	0	0/0
 18	Haylee Blinkhorn	 2	0	0	0	0/0
 13	An

In [7]:
fileName=('SeasonSkaterStatsWomens.txt')
jerseyList=[]
with open(fileName, 'r', encoding='utf-8') as f:
    read_data = f.read()
    rows=read_data.split('\n')
    for i in rows:
        col=i.split('\t')
        jerseyDict={'number':int(col[0]),
                    'player':col[1],
                    'season':col[-1]}
        jerseyList.append(jerseyDict)           

In [8]:
dfJersey=pd.DataFrame(jerseyList)

In [102]:
dfJersey['season']=dfJersey.groupby(['number','player'])['season'].transform(lambda x: ','.join(x))
dfJersey[['number','player','season']].drop_duplicates()

,number,player,season
0,17,Laurel Koller,"2005-06,2007-08,2008-09,2009-10"
1,9,Gina Kearns,"2005-06,2006-07,2007-08,2008-09"
2,22,Nicki Wiart,"2005-06,2006-07,2007-08,2008-09"
3,8,Cara Hendry,"2005-06,2006-07"
4,10,Erin Seman,"2005-06,2006-07,2007-08,2008-09"
...,...,...,...
385,10,Jesse DeVito,2021-22
389,14,Madison Michals,2021-22
390,15,Ellie Larson,2021-22
397,22,Christina Vote,2021-22


In [106]:
dfJersey=dfJersey.sort_values('number').drop_duplicates()

In [5]:
fileName=('JerseyNumbersWomens.txt')
jerseyList=[]
with open(fileName, 'r', encoding='utf-8') as f:
    read_data = f.read()
    rows=read_data.split('\n')
    for i in rows:
        col=i.split('\t')
        jerseyDict={'number':int(col[0]),
                    'player':col[1],
                    'season':col[-1]}
        jerseyList.append(jerseyDict)
dfJerseyWomens=pd.DataFrame(jerseyList)

In [6]:
dfJerseyWomens

,number,player,season
0,1,Kate Stuart,"2018-19,2019-20,2020-21,2021-22"
1,1,Kerrin Sperry,"2010-11,2011-12,2012-13,2013-14"
2,1,Ashley Leichliter,"2007-08,2008-09"
3,1,Ashlyn Aiello,"2015-16,2017-18"
4,1,Eden Spencer,2005-06
...,...,...,...
134,33,Melissa Haber,"2006-07,2007-08,2008-09,2009-10"
135,33,Victoria Hanson,"2013-14,2014-15,2015-16,2016-17"
136,36,Alissa Fromkin,2012-13
137,37,Callie Shanahan,2021-22


In [7]:
fileNameW=('SeasonSkaterStatsWomens.txt')
seasList=[]
with open(fileNameW, 'r', encoding='utf-8') as f:
    read_data = f.read()
    rows=read_data.split('\n')
    for i in rows:
        col=i.split('\t')
        seasDict={'number':col[0],
                 'name':col[1],
                 'pos':col[2],
                 'yr':col[3],
                 'gp':int(col[4]),
                 'goals':int(col[5]),
                 'assists':int(col[6]),
                 'pts':int(col[7]),
                 'pens':col[8],
                 'season':col[9],
                 'year':int(col[9][:4])+1}
        seasList.append(seasDict)
f.close()
dfSeasSkateW=pd.DataFrame(seasList)

In [37]:
dfSeasSkateW[['pin','pim']]=dfSeasSkateW['pens'].str.split('/',expand=True)

In [9]:
seas=list(dfSeasSkateW['season'].unique())

In [18]:
for season in seas:
    goals=dfSeasSkateW.loc[dfSeasSkateW['season']==season].nlargest(1,'goals',keep='all')[['goals','name']].to_string(index=False).lstrip(' goals           name\n').lstrip().rstrip('\n')
    assists=dfSeasSkateW.loc[dfSeasSkateW['season']==season].nlargest(1,'assists',keep='all')[['assists','name']].to_string(index=False).lstrip(' asssits           name\n').lstrip().rstrip('\n')
    pts=dfSeasSkateW.loc[dfSeasSkateW['season']==season].nlargest(1,'pts',keep='all')[['pts','name']].to_string(index=False).lstrip(' pts           name\n').lstrip().rstrip('\n')
    print(season,goals,assists,pts)

2005-06 13  Laurel Koller
    13    Gina Kearns 15  Laurel Koller 28  Laurel Koller
2006-07 19  Erin Seman 22  Melissa Anderson 34  Melissa Anderson
2007-08 16  Melissa Anderson 16       Gina Kearns
      16  Jillian Kirchner 29  Melissa Anderson
2008-09 15  Jenelle Kohanchuk
    15        Gina Kearns 22  Melissa Anderson 31  Jenelle Kohanchuk
2009-10 21  Melissa Anderson 18  Melissa Anderson 39  Melissa Anderson
2010-11 32  Jenn Wakefield 27  Catherine Ward 54  Jenn Wakefield
2011-12 29  Jenn Wakefield 30  Isabel Menard 57  Jenn Wakefield
2012-13 25  Jenelle Kohanchuk 36  Marie-Philip Poulin 55  Marie-Philip Poulin
2013-14 32  Sarah Lefort 26  Kaleigh Fratkin 55  Sarah Lefort
2014-15 27  Marie-Philip Poulin 27  Marie-Philip Poulin
      27         Sarah Lefort 54  Marie-Philip Poulin
2015-16 22  Victoria Bach 34  Rebecca Leslie 49  Rebecca Leslie
2016-17 24  Mary Parker 26  Victoria Bach 49  Victoria Bach
2017-18 39  Victoria Bach 33  Rebecca Leslie 67  Victoria Bach
2018-19 25  Sammy

In [28]:
dfSeasSkateW.loc[dfSeasSkateW['season']=='2014-15'].nlargest(1,'assists',keep='all')[['assists','name']]

,assists,name
215,27,Marie-Philip Poulin
216,27,Sarah Lefort


In [41]:
dfSeasSkateW['pin']=dfSeasSkateW['pin'].astype('int')
dfSeasSkateW['pim']=dfSeasSkateW['pim'].astype('int')

In [113]:
df=dfSeasSkateW.groupby('name').sum()

In [114]:
dfA=dfSeasSkateW.groupby('name')['season'].apply(','.join).reset_index()

In [115]:
x=dfA.to_string().split('\n')[1:]

In [117]:
x=dfA.to_string().split('\n')[1:]
s=[]
for i in x:
    d=list(filter(None,i.split('  ')))
    d[2]=d[2][0:4]+'-'+d[2][-2:]
    s.append(d[2])
df['season']=s

In [118]:
df['season']=s

In [104]:
x="2017-18,2018-19,2019-20"

In [110]:
x[0:4]+'-'+x[-2:]

'2017-20'

In [125]:
for i in df[['season','gp','goals','assists','pts','pin','pim']].to_string().split('\n'):
    row=list(filter(None,i.split('  ')))
    print("\t".join(row))

 season	 gp	goals	assists	pts	pin	pim
name	 
Abbey Stanley	 2017-20	110	5	 40	 45	 35	 70
Abby Cook	 2016-20	147	 24	 75	 99	 45	 90
Alanna Sosenko	2005-06	1	0	0	0	0	0
Alex Allan	2018-22	118	4	 20	 24	 34	 71
Alexandra Calderone	 2016-20	 91	0	9	9	 13	 26
Alexis Crossley	 2015-17	 76	 12	 29	 41	 18	 36
Alexis Woloschuk	2012-16	114	1	 23	 24	 18	 36
Alissa Fromkin	2009-13	 28	0	0	0	2	4
Allyse Wilcox	 2005-09	 98	0	0	0	 10	 20
Amanda Shaw	 2005-09	136	 12	 29	 41	 78	164
Amy Ley	 2005-06	3	0	0	0	0	0
Andi Calderone	2020-22	 45	0	 10	 10	5	 10
Anna Streifel	 2015-17	 24	1	0	1	0	0
Anya Battaglino	 2010-12	 18	0	0	0	2	4
Ashley Leichliter	 2007-09	4	0	0	0	0	0
Ashley Watkins	2005-06	 10	0	0	0	2	4
Ashlyn Aiello	 2015-18	3	0	0	0	0	0
Braly Hiller	2010-12	5	0	0	0	0	0
Breanna Scarpaci	2016-20	138	3	 21	 24	 21	 42
Britt Hergesheimer	2009-13	110	2	3	5	 18	 36
Brooke Ersoy	2019-22	 34	0	3	3	4	8
Caitlin Reddy	 2005-09	105	0	5	5	 23	 46
Callie Shanahan	 2021-22	 22	0	0	0	0	0
Cara Hendry	 2005-07	 65	 

In [126]:
fileName=('GoalieStatsWomens.txt')
goalieList=[]
with open(fileName, 'r', encoding='utf-8') as f:
    read_data = f.read()
    rows=read_data.split('\n')
    for i in rows:
        goalieSearch=re.search('(.*),(.*) (\S*) (\S*) (\S*) (\S*) (\S*) (\S*) (\S*) (\S*) (\S*) (\S*)',i)
        if goalieSearch!=None:
            mins=goalieSearch.group(5).split(':')
            if(len(mins)>1):
                time = "{}:{}".format(*divmod(int(mins[0]), 60)) + ":" + mins[1]
                time = pd.to_timedelta(time)
            else:
                time=float('nan')
            goalieDict={'last':goalieSearch.group(1),
                       'first':goalieSearch.group(2),
                       'name':goalieSearch.group(2)+' '+goalieSearch.group(1),
                       'seasons':convertSeasons(goalieSearch.group(3)),
                       'gp':convertToInt(goalieSearch.group(4)),
                       'mins':round(pd.Timedelta(time).total_seconds()/60,2),
                       'ga':convertToInt(goalieSearch.group(6)),
                       'gaa':convertToFloat(goalieSearch.group(7)),
                       'saves':convertToInt(goalieSearch.group(8)),
                       'sv%':convertToFloat(goalieSearch.group(9)),
                       'W':convertToInt(goalieSearch.group(10)),
                       'L':convertToInt(goalieSearch.group(11)),
                       'T':convertToInt(goalieSearch.group(12))}
            goalieList.append(goalieDict)
dfGoalie=pd.DataFrame(goalieList)

FileNotFoundError: [Errno 2] No such file or directory: 'GoalieStatsWomens.txt'

In [127]:
fileNameW=('SeasonGoalieStatsWomens.txt')
with open(fileNameW, 'r', encoding='utf-8') as f:
    read_data = f.read()
    rows=read_data.split('\n')
    seasGList=[]
    for i in rows:
        col=i.split('\t')
        seasGDict={'number':col[0],
                 'name':col[1],
                 'yr':col[2],
                 'gp':int(col[3]),
                 'mins':col[4],
                 'ga':int(col[5]),
                 'saves':int(col[6]),
                 'sv%':float(col[7]),
                 'gaa':float(col[8]),
                 'record':col[9],
                 'SO':int(col[10]),
                 'season':col[11],
                 'year':int(col[11][:4])+1}
        seasGList.append(seasGDict)
f.close()
dfSeasGoalie=pd.DataFrame(seasGList)

In [131]:
dfTot['gp']=dfSeasGoalie.groupby('name')['gp'].sum()

NameError: name 'dfTot' is not defined

In [218]:
minsSplit=dfSeasGoalie['mins'].str.split(':',expand=True)
dfSeasGoalie[['w','l','t']]=dfSeasGoalie['record'].str.split('-',expand=True)
dfSeasGoalie['min']=minsSplit[0].astype('int')
dfSeasGoalie['sec']=minsSplit[1].astype('int')

dfSeasGoalie['w']=dfSeasGoalie['w'].astype('int')
dfSeasGoalie['l']=dfSeasGoalie['l'].astype('int')
dfSeasGoalie['t']=dfSeasGoalie['t'].astype('int')
dfTot=dfSeasGoalie.groupby('name').sum()
dfA=dfSeasGoalie.groupby('name')['season'].apply(','.join).reset_index()
x=dfA.to_string().split('\n')[1:]
s=[]
for i in x:
    d=list(filter(None,i.split('  ')))
    d[2]=d[2][0:4]+'-'+d[2][-2:]
    s.append(d[2])
dfTot['season']=s
dfTot['sv%']=round(dfTot['saves']/(dfTot['ga']+dfTot['saves']),3)
dfTot['min']=dfTot['min']+dfTot['sec']//60
dfTot['sec']%=60
dfTot['gaa']=round(dfTot['ga']*60/dfTot['min'],2)

In [223]:
for i in dfTot[['season','gp','min','sec','ga','gaa','saves','sv%','w','l','t']].to_string().split('\n'):
    row=list(filter(None,i.split('  ')))
    print("\t".join(row))

season	 gp	 min	sec	 ga	 gaa	saves	sv%	 w	 l	 t
name	 
Alissa Fromkin	 2009-13	 28	1419	6	 58	2.45	470	0.890	13	 5	 5
Allyse Wilcox	2005-09	 98	5736	 38	222	2.32	 2418	0.916	43	43	11
Ashley Leichliter	2007-09	4	46	 38	2	2.61	9	0.818	 0	 0	 0
Ashlyn Aiello	2015-18	3	18	 33	0	0.00	 11	1.000	 0	 0	 0
Braly Hiller	 2010-12	5	 237	 41	9	2.28	 98	0.916	 3	 0	 0
Callie Shanahan	2021-22	 22	 834	 59	 33	2.37	406	0.925	 6	 6	 2
Corinne Schroeder	2017-21	 91	5391	 37	178	1.98	 2316	0.929	49	27	14
Eden Spencer	 2005-06	1	 3	3	0	0.00	2	1.000	 0	 0	 0
Erin O'Neil	2014-18	 69	3894	 15	189	2.91	 1858	0.908	32	24	 7
Karilyn Pilch	2005-06	8	 395	 52	 24	3.65	184	0.885	 3	 3	 0
Kate Stuart	2018-22	 42	2235	 44	 73	1.96	983	0.931	20	10	 6
Kerrin Sperry	2010-14	127	7612	 50	283	2.23	 3329	0.922	85	34	 8
Melissa Haber	2006-10	 69	4069	 11	152	2.24	 1614	0.914	33	19	14
Mia Becker	 2014-15	2	23	4	2	5.22	 12	0.857	 0	 0	 0
Nicole McGuigan	2018-22	 17	 139	 55	4	1.73	 51	0.927	 0	 2	 0
Victoria Hanson	2013-17	

In [224]:
dfTot

,gp,ga,saves,sv%,gaa,SO,year,min,sec,w,l,t,season
name,,,,,,,,,,,,,
Alissa Fromkin,28,58,470,0.890,2.45,23,8046,1419,6,13,5,5,2009-13
Allyse Wilcox,98,222,2418,0.916,2.32,48,8030,5736,38,43,43,11,2005-09
Ashley Leichliter,4,2,9,0.818,2.61,0,4017,46,38,0,0,0,2007-09
Ashlyn Aiello,3,0,11,1.000,0.00,0,4034,18,33,0,0,0,2015-18
Braly Hiller,5,9,98,0.916,2.28,3,4023,237,41,3,0,0,2010-12
Callie Shanahan,22,33,406,0.925,2.37,1,2022,834,59,6,6,2,2021-22
Corinne Schroeder,91,178,2316,0.929,1.98,8,8078,5391,37,49,27,14,2017-21
Eden Spencer,1,0,2,1.000,0.00,0,2006,3,3,0,0,0,2005-06
Erin O'Neil,69,189,1858,0.908,2.91,3,8066,3894,15,32,24,7,2014-18


In [20]:
fileName=('BeanpotHistoryWomens.txt')
with open(fileName, 'r', encoding='utf-8') as f:
    read_data = f.read()
    rows=read_data.split('\n')
    beanList=[]
    for i in rows:
        col=i.split('\t')
        beanDict={  "edition" : int(col[0]),
                    "year" : int(col[1]),
                    "arena" : col[2],
                    "semiDate" : col[3],
                    "note": col[4],
                    "semi1Winner" : col[5],
                    "semi1WinnerScore" : int(col[6]),
                    "semi1Loser" : col[7],
                    "semi1LoserScore" : int(col[8]),
                    "semi1OT" : col[9],
                    "semi2Winner" : col[10],
                    "semi2WinnerScore" : int(col[11]),
                    "semi2Loser" : col[12],
                    "semi2LoserScore" : int(col[13]),
                    "semi2OT" : col[14],
                    "semi3Winner" : col[15],
                    "semi3WinnerScore" : convertToInt(col[16]),
                    "semi3Loser" : col[17],
                    "semi3LoserScore" : convertToInt(col[18]),
                    "champDate" : col[19],
                    "consWinner" : col[20],
                    "consWinnerScore" : convertToInt(col[21]),
                    "consLoser" : col[22],
                    "consLoserScore" : convertToInt(col[23]),
                    "consOT" : col[24],
                    "champion" : col[25],
                    "championScore" : int(col[26]),
                    "runnerup" : col[27],
                    "runnerupScore" : int(col[28]),
                    "champOT" : col[29]}
        beanDict['semiDate']=beanDict['semiDate'].rstrip(' ').lstrip(' ')
        beanDict['champDate']=beanDict['champDate'].rstrip(' ').lstrip(' ')
        beanDict['semiDate']+='/'+str(beanDict['year'])
        beanDict['champDate']+='/'+str(beanDict['year'])
        beanDict['semiDate']=pd.Timestamp(beanDict['semiDate'])
        beanDict['champDate']=pd.Timestamp(beanDict['champDate'])
        beanDict['semiDOW']=beanDict['semiDate'].weekday()
        beanDict['champDOW']=beanDict['champDate'].weekday()
        beanDict['semi1GD']=beanDict['semi1WinnerScore']-beanDict['semi1LoserScore']
        beanDict['semi2GD']=beanDict['semi2WinnerScore']-beanDict['semi2LoserScore']
        beanDict['consGD']=beanDict['consWinnerScore']-beanDict['consLoserScore']
        beanDict['champGD']=beanDict['championScore']-beanDict['runnerupScore']
        beanList.append(beanDict)
f.close()
dfBeanpot=pd.DataFrame(beanList)

In [10]:
import pandas as pd

In [18]:
def convertToInt(val):
        if(val.isdigit()):
            val=int(val)
        else:
            val=0
        return val

In [21]:
dfBeanpot

,edition,year,arena,semiDate,note,semi1Winner,semi1WinnerScore,semi1Loser,semi1LoserScore,semi1OT,...,championScore,runnerup,runnerupScore,champOT,semiDOW,champDOW,semi1GD,semi2GD,consGD,champGD
0,1,1979,Northeastern,1979-03-16,,Boston College,4,Boston University,1,,...,3,Boston College,1,,4,5,3,4,3,2
1,2,1980,Harvard,1980-02-23,,Northeastern,11,Boston College,0,,...,7,Harvard,1,,5,6,11,2,1,6
2,3,1981,Northeastern,1981-02-17,,Boston College,2,Harvard,1,,...,4,Boston College,0,,1,4,1,4,8,4
3,4,1982,Boston University,1982-02-16,,Harvard,3,Northeastern,2,(5OT),...,2,Boston University,1,,1,4,1,10,3,1
4,5,1983,Harvard,1983-02-08,,Harvard,12,Boston College,0,,...,2,Northeastern,1,,1,4,12,10,10,1
5,6,1984,Northeastern,1984-02-07,,Northeastern,14,Boston College,0,,...,7,Harvard,1,,1,4,14,10,1,6
6,7,1985,Harvard,1985-02-05,,Boston College,4,Boston University,0,,...,7,Boston College,0,,1,1,4,6,7,7
7,8,1986,Northeastern,1986-02-05,,Northeastern,6,Boston University,0,,...,7,Harvard,0,,2,1,6,11,2,7
8,9,1987,Harvard,1987-02-03,,Northeastern,10,Boston College,0,,...,7,Harvard,1,,1,1,10,10,1,6
9,10,1988,Harvard,1988-02-02,,Northeastern,2,Boston University,0,,...,5,Harvard,1,,1,1,2,11,1,4


In [32]:
import urllib.request, urllib.error, urllib.parse
from bs4 import BeautifulSoup
import re
seasons=["2005-06","2006-07","2007-08","2008-09","2009-10","2010-11","2011-12","2012-13","2013-14","2014-15","2015-16","2016-17","2017-18","2018-19","2019-20","2020-21"]
playList=[]
goalieList=[]
for season in seasons:
    sSeason=season[2:4]+season[-2:]
    url = "http://www.collegehockeystats.net/{}/rosters/bu_w".format(sSeason)
    #print(url)
    f=urllib.request.urlopen(url)
    html = f.read()
    f.close()
    soup = BeautifulSoup(html, 'html.parser')
    table= soup.find_all('table')[1]
    rows=table.find_all('tr')
    for i in rows:
        if('Name' in i.get_text()):
            continue
        col=i.find_all('td')
        pDict={'number':col[0].get_text().strip('#'),
              'name':col[1].get_text().replace('\xa0',' ').strip(),
              'yr':col[2].get_text().replace('\xa0',''),
              'pos':col[3].get_text(),
              'height':col[4].get_text(),
              'shot':col[5].get_text(),
              'yob':col[6].get_text().replace('\xa0',''),
              'prev':col[7].get_text(),
              'season':season}
        if(pDict['shot']==''):
            pDict['shot']=='N/A'
        '''
        if(pDict['yob']==''):
            pass
        elif(int(pDict['yob'])>20):
            pDict['yob']=int('19'+ (pDict['yob']))
        else:
            pDict['yob']=int('20'+ (pDict['yob']))
        pDict['h_feet'],pDict['h_in']=pDict['height'].split('-')
        pDict['home'],pDict['prevTeam']=pDict['prev'].split('/')
        pDict['hometown'],pDict['homeState']=pDict['home'].split(',')
        if(pDict['yob']!=''):
            pDict['age']=int(season[:4])-pDict['yob']
        nhlSearch=re.search('(\(.*\))',pDict['name'])
        if(nhlSearch!=None):
            pDict['name']=pDict['name'].replace(nhlSearch.group(1),'').strip()
            pDict['nhl']=nhlSearch.group(1).strip('(').strip(')')
        else:
            pDict['nhl']=''
        pDict['season']=season
        del pDict['prev']
        '''
        playList.append(pDict)
dfPlayData=pd.DataFrame(playList)

In [35]:
dfPlayData.loc[dfPlayData['yob']=='','yob']='N/A'

In [36]:
f=open('PlayerDataWomens.txt','w')
for i in dfPlayData.to_string(index=False,header=False).split('\n'):
    row=list(filter(None,i.split('  ')))
    print("\t".join(row),file=f)
f.close()